## Importing libraries 

In [1]:
import time
import pandas as pd
import numpy as np
import copy as cp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.neighbors import NearestNeighbors

## Bases importing

In [2]:
qa_base = pd.read_csv('./insurance_qna_dataset.csv',sep='\t',index_col=0)
questions_number = qa_base.shape[0]

qm_base = pd.read_csv("Test questions dataset.csv")
test_questions_number = qm_base.shape[0]

# qa_base.head()
qm_base.head(21)

,original question,modified question – light,modified question – medium,modified question – heavy
0,How Many People Go Without Health Insurance?,How Many People Live Without Health Insurance?,How Many People Lack Health Coverage?,How Many Individuals Lack Good Health Coverage?
1,What Is The Purpose Of A Life Insurance Policy?,What Is The Point Of A Life Insurance Policy?,What Serves As A Life Insurance Policy's Objec...,What Do Life Insurance Policies Serve?
2,How Much Is A Typical Homeowners Insurance?,How Much Is An Average Homeowners Insurance?,How Much Does The Average Homeowners Insurance...,What Is the Average Cost Of Homeowners Insurance?
3,Is Disability Insurance Worth Having?,Is Disability Insurance Good?,Is Disability Insurance Worth Enough To Have It?,What Is The Value of Disability Insurance?
4,Is Vision Covered Under Medicare?,Is Vision Not Covered Under Medicare?,Does Medicare Cover Vision Care?,Is Medicare Going To Cover Vision Care?
5,Does Life Insurance Cover Both Spouses?,Does Life Insurance Cover Both Spouse?,Does Life Insurance Cover Both Partners?,Are Both Spouses Covered By Life Insurance?
6,Can Bad Credit Affect Car Insurance?,Can Bad Credit Affect Cars Insurance?,Can Bad Credits Affects Cars Insurance?,Can A Poor Credit History Affect Auto Insurance?
7,How Much Should I Get Life Insurance For?,How Much Should I I I Get Life Insurance For?,How Long Should I Get Life Insurance For?,How Much Life Insurance Should I Purchase?
8,Who Underwrites Hsbc Life Insurance?,Who Underwrite Hsbc Life Insurances?,Who Underwrites Hsbbcc Life Insurances?,Who Drafts The Hsbc Life Insurance Policy?
9,How Much Is Home Insurance Per Month?,How Much Is Home Insurance Per Months?,How Much Is House Insurance Per Month?,How Much Does Monthly Home Insurance Cost?


## Recording results

In [3]:
results = pd.DataFrame(columns = ['method',
                                  'word simplification',
                                  'questions modification',
                                  'metric',
                                  'average sentence rank',
                                  'average execution time [s]'])

## TF-IDF vectorization - loop method

In [4]:
vectorizer = TfidfVectorizer()
qa_base_vectorized = vectorizer.fit_transform(qa_base["Question"])

qlm_vectorized = vectorizer.transform(qm_base["modified question – light"])
qmm_vectorized = vectorizer.transform(qm_base["modified question – medium"])
qhm_vectorized = vectorizer.transform(qm_base["modified question – heavy"])

### Lightly modified questions - loop method

In [5]:
euclidean_rank = []
manhattan_rank = []
cos_rank = []

start = time.time()

for i in range(test_questions_number):

    original_question = qm_base["original question"][i]
    modified_question = qlm_vectorized[i]
    
    distance_euclidean = pairwise_distances(modified_question, qa_base_vectorized, metric='euclidean')
    distance_manhattan = pairwise_distances(modified_question, qa_base_vectorized, metric='manhattan')
    cos_similarity = cosine_similarity(modified_question, qa_base_vectorized)
    
    distance_euclidean_indexes = np.argsort(distance_euclidean)
    distance_manhattan_indexes = np.argsort(distance_manhattan)
    cos_similarity_indexes = np.argsort(cos_similarity)
  
    for j in range(questions_number):
        ranked_question = qa_base["Question"][distance_euclidean_indexes[0][j]]
        if original_question == ranked_question:
            euclidean_rank.append(j+1)
            break
            
    for j in range(questions_number):
        ranked_question = qa_base["Question"][distance_manhattan_indexes[0][j]]
        if original_question == ranked_question:
            manhattan_rank.append(j+1)
            break
            
    for j in range(questions_number):
        ranked_question = qa_base["Question"][cos_similarity_indexes[0][::-1][j]]
        if original_question == ranked_question:
            cos_rank.append(j+1)
            break
            
end = time.time()
t = (end-start)/test_questions_number
            
print("Light modifications - euclidean rank:", euclidean_rank)
print("Light modifications - manhattan rank:", manhattan_rank)
print("Light modifications - cos rank:", cos_rank)

data = {'method':'loop',
         'word simplification':'/',
         'questions modification':'light',
         'metric':'euclidean',
         'average sentence rank':round(np.mean(euclidean_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'loop',
         'word simplification':'/',
         'questions modification':'light',
         'metric':'manhattan',
         'average sentence rank':round(np.mean(manhattan_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'loop',
         'word simplification':'/',
         'questions modification':'light',
         'metric':'cosine',
         'average sentence rank':round(np.mean(cos_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

Light modifications - euclidean rank: [1, 14, 122, 703, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 5, 31, 1, 1, 1, 303]
Light modifications - manhattan rank: [1, 5, 26, 263, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 14, 1, 1, 1, 80]
Light modifications - cos rank: [1, 14, 122, 703, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 5, 31, 1, 1, 1, 303]


/tmp/ipykernel_43013/3679028540.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/3679028540.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/3679028540.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)


### Medium modified questions - loop method

In [6]:
euclidean_rank = []
manhattan_rank = []
cos_rank = []

start = time.time()

for i in range(test_questions_number):

    original_question = qm_base["original question"][i]
    modified_question = qmm_vectorized[i]
    
    distance_euclidean = pairwise_distances(modified_question, qa_base_vectorized, metric='euclidean')
    distance_manhattan = pairwise_distances(modified_question, qa_base_vectorized, metric='manhattan')
    cos_similarity = cosine_similarity(modified_question, qa_base_vectorized)
    
    distance_euclidean_indexes = np.argsort(distance_euclidean)
    distance_manhattan_indexes = np.argsort(distance_manhattan)
    cos_similarity_indexes = np.argsort(cos_similarity)
  
    for j in range(questions_number):
        ranked_question = qa_base["Question"][distance_euclidean_indexes[0][j]]
        if original_question == ranked_question:
            euclidean_rank.append(j+1)
            break
            
    for j in range(questions_number):
        ranked_question = qa_base["Question"][distance_manhattan_indexes[0][j]]
        if original_question == ranked_question:
            manhattan_rank.append(j+1)
            break
            
    for j in range(questions_number):
        ranked_question = qa_base["Question"][cos_similarity_indexes[0][::-1][j]]
        if original_question == ranked_question:
            cos_rank.append(j+1)
            break
            
end = time.time()
t = (end-start)/test_questions_number
            
print("Medium modifications - euclidian rank:", euclidean_rank)
print("Medium modifications - manhattan rank:", manhattan_rank)
print("Medium modifications - cos rank:", cos_rank)

data = {'method':'loop',
         'word simplification':'/',
         'questions modification':'medium',
         'metric':'euclidean',
         'average sentence rank':round(np.mean(euclidean_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'loop',
         'word simplification':'/',
         'questions modification':'medium',
         'metric':'manhattan',
         'average sentence rank':round(np.mean(manhattan_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'loop',
         'word simplification':'/',
         'questions modification':'medium',
         'metric':'cosine',
         'average sentence rank':round(np.mean(cos_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

Medium modifications - euclidian rank: [6, 307, 667, 101, 11, 2, 21, 1, 3, 12, 1, 69, 1, 1, 1, 1, 5, 1, 106, 1, 161]
Medium modifications - manhattan rank: [4, 423, 248, 36, 68, 1, 15, 1, 1, 11, 1, 55, 1, 1, 1, 1, 1, 1, 114, 6, 60]
Medium modifications - cos rank: [6, 307, 667, 101, 11, 2, 21, 1, 3, 12, 1, 69, 1, 1, 1, 1, 5, 1, 106, 1, 161]


/tmp/ipykernel_43013/519489947.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/519489947.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/519489947.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)


### Heavily modified questions - loop method

In [7]:
euclidean_rank = []
manhattan_rank = []
cos_rank = []

start = time.time()

for i in range(test_questions_number):

    original_question = qm_base["original question"][i]
    modified_question = qhm_vectorized[i]
    
    distance_euclidean = pairwise_distances(modified_question, qa_base_vectorized, metric='euclidean')
    distance_manhattan = pairwise_distances(modified_question, qa_base_vectorized, metric='manhattan')
    cos_similarity = cosine_similarity(modified_question, qa_base_vectorized)
    
    distance_euclidean_indexes = np.argsort(distance_euclidean)
    distance_manhattan_indexes = np.argsort(distance_manhattan)
    cos_similarity_indexes = np.argsort(cos_similarity)
  
    for j in range(questions_number):
        ranked_question = qa_base["Question"][distance_euclidean_indexes[0][j]]
        if original_question == ranked_question:
            euclidean_rank.append(j+1)
            break
            
    for j in range(questions_number):
        ranked_question = qa_base["Question"][distance_manhattan_indexes[0][j]]
        if original_question == ranked_question:
            manhattan_rank.append(j+1)
            break
            
    for j in range(questions_number):
        ranked_question = qa_base["Question"][cos_similarity_indexes[0][::-1][j]]
        if original_question == ranked_question:
            cos_rank.append(j+1)
            break
            
end = time.time()
t = (end-start)/test_questions_number

print("Heavy modifications - euclidian rank:", euclidean_rank)
print("Heavy modifications - manhattan rank:", manhattan_rank)
print("Heavy modifications - cos rank:", cos_rank)

data = {'method':'loop',
         'word simplification':'/',
         'questions modification':'heavy',
         'metric':'euclidean',
         'average sentence rank':round(np.mean(euclidean_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'loop',
         'word simplification':'/',
         'questions modification':'heavy',
         'metric':'manhattan',
         'average sentence rank':round(np.mean(manhattan_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'loop',
         'word simplification':'/',
         'questions modification':'heavy',
         'metric':'cosine',
         'average sentence rank':round(np.mean(cos_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

Heavy modifications - euclidian rank: [17, 2748, 1443, 1498, 7, 1, 19, 32, 2, 610, 21, 461, 1, 6, 299, 1, 5, 1, 30, 1, 28]
Heavy modifications - manhattan rank: [19, 4180, 801, 630, 5, 1, 15, 38, 1, 454, 33, 803, 1, 3, 628, 1, 1, 1, 2, 1, 13]
Heavy modifications - cos rank: [17, 2748, 1443, 1498, 7, 1, 19, 32, 2, 610, 21, 461, 1, 6, 299, 1, 5, 1, 30, 1, 28]


/tmp/ipykernel_43013/2722385614.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/2722385614.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/2722385614.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)


## TF-IDF vectorization with stemming - loop method

In [8]:
qa_base_stem = cp.deepcopy(qa_base["Question"])
ps = PorterStemmer()

for i in range(questions_number):
    sentence = qa_base_stem[i]
    token_words=word_tokenize(sentence)
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(ps.stem(word))
        stem_sentence.append(" ")
        stemmed_sentence = "".join(stem_sentence)
    qa_base_stem.iloc[i] = stemmed_sentence

vectorizer = TfidfVectorizer()
qa_base_vectorized = vectorizer.fit_transform(qa_base_stem)

qlm_vectorized = vectorizer.transform(qm_base["modified question – light"])
qmm_vectorized = vectorizer.transform(qm_base["modified question – medium"])
qhm_vectorized = vectorizer.transform(qm_base["modified question – heavy"])

### Lightly modified questions with stemming - loop method

In [9]:
euclidean_rank = []
manhattan_rank = []
cos_rank = []

start = time.time()

for i in range(test_questions_number):
    original_question = qm_base["original question"][i]
    modified_question = qlm_vectorized[i]
    
    distance_euclidean = pairwise_distances(modified_question, qa_base_vectorized, metric='euclidean')
    distance_manhattan = pairwise_distances(modified_question, qa_base_vectorized, metric='manhattan')
    cos_similarity = cosine_similarity(modified_question, qa_base_vectorized)
    
    distance_euclidean_indexes = np.argsort(distance_euclidean)
    distance_manhattan_indexes = np.argsort(distance_manhattan)
    cos_similarity_indexes = np.argsort(cos_similarity)
  
    for j in range(questions_number):
        ranked_question = qa_base["Question"][distance_euclidean_indexes[0][j]]
        if original_question == ranked_question:
            euclidean_rank.append(j+1)
            break
            
    for j in range(questions_number):
        ranked_question = qa_base["Question"][distance_manhattan_indexes[0][j]]
        if original_question == ranked_question:
            manhattan_rank.append(j+1)
            break
            
    for j in range(questions_number):
        ranked_question = qa_base["Question"][cos_similarity_indexes[0][::-1][j]]
        if original_question == ranked_question:
            cos_rank.append(j+1)
            break
            
end = time.time()
t = (end-start)/test_questions_number

print("Light modifications - euclidean rank:", euclidean_rank)
print("Light modifications - manhattan rank:", manhattan_rank)
print("Light modifications - cos rank:", cos_rank)

data = {'method':'loop',
         'word simplification':'stemming',
         'questions modification':'light',
         'metric':'euclidean',
         'average sentence rank':round(np.mean(euclidean_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'loop',
         'word simplification':'stemming',
         'questions modification':'light',
         'metric':'manhattan',
         'average sentence rank':round(np.mean(manhattan_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'loop',
         'word simplification':'stemming',
         'questions modification':'light',
         'metric':'cosine',
         'average sentence rank':round(np.mean(cos_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

Light modifications - euclidean rank: [24, 42, 333, 1204, 1, 1, 1, 1, 1, 1, 2, 1, 30, 154, 1, 1, 31, 1, 88, 15, 1052]
Light modifications - manhattan rank: [11, 12, 158, 1361, 1, 1, 1, 1, 1, 1, 2, 1, 22, 82, 3, 1, 16, 1, 63, 32, 316]
Light modifications - cos rank: [24, 42, 333, 1204, 1, 1, 1, 1, 1, 1, 2, 1, 30, 154, 1, 1, 31, 1, 88, 15, 1052]


/tmp/ipykernel_43013/313608173.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/313608173.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/313608173.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)


### Medium modified questions with stemming - loop method

In [10]:
euclidean_rank = []
manhattan_rank = []
cos_rank = []

start = time.time()

for i in range(test_questions_number):

    original_question = qm_base["original question"][i]
    modified_question = qmm_vectorized[i]
    
    distance_euclidean = pairwise_distances(modified_question, qa_base_vectorized, metric='euclidean')
    distance_manhattan = pairwise_distances(modified_question, qa_base_vectorized, metric='manhattan')
    cos_similarity = cosine_similarity(modified_question, qa_base_vectorized)
    
    distance_euclidean_indexes = np.argsort(distance_euclidean)
    distance_manhattan_indexes = np.argsort(distance_manhattan)
    cos_similarity_indexes = np.argsort(cos_similarity)
  
    for j in range(questions_number):
        ranked_question = qa_base["Question"][distance_euclidean_indexes[0][j]]
        if original_question == ranked_question:
            euclidean_rank.append(j+1)
            break
            
    for j in range(questions_number):
        ranked_question = qa_base["Question"][distance_manhattan_indexes[0][j]]
        if original_question == ranked_question:
            manhattan_rank.append(j+1)
            break
            
    for j in range(questions_number):
        ranked_question = qa_base["Question"][cos_similarity_indexes[0][::-1][j]]
        if original_question == ranked_question:
            cos_rank.append(j+1)
            break
            
end = time.time()
t = (end-start)/test_questions_number
            
print("Medium modifications - euclidian rank:", euclidean_rank)
print("Medium modifications - manhattan rank:", manhattan_rank)
print("Medium modifications - cos rank:", cos_rank)

data = {'method':'loop',
         'word simplification':'stemming',
         'questions modification':'medium',
         'metric':'euclidean',
         'average sentence rank':round(np.mean(euclidean_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'loop',
         'word simplification':'stemming',
         'questions modification':'medium',
         'metric':'manhattan',
         'average sentence rank':round(np.mean(manhattan_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'loop',
         'word simplification':'stemming',
         'questions modification':'medium',
         'metric':'cosine',
         'average sentence rank':round(np.mean(cos_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

Medium modifications - euclidian rank: [397, 1224, 1398, 22, 4, 1, 9, 1, 737, 12, 2, 1, 182, 288, 3, 1, 5, 1, 410, 46, 764]
Medium modifications - manhattan rank: [672, 3440, 730, 28, 4, 1, 6, 1, 216, 12, 2, 1, 101, 426, 8, 1, 1, 1, 363, 357, 267]
Medium modifications - cos rank: [397, 1224, 1398, 22, 4, 1, 9, 1, 737, 12, 2, 1, 182, 288, 3, 1, 5, 1, 410, 46, 764]


/tmp/ipykernel_43013/3961608353.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/3961608353.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/3961608353.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)


### Heavily modified questions with stemming - loop method

In [11]:
euclidean_rank = []
manhattan_rank = []
cos_rank = []

start = time.time()

for i in range(test_questions_number):

    original_question = qm_base["original question"][i]
    modified_question = qhm_vectorized[i]
    
    distance_euclidean = pairwise_distances(modified_question, qa_base_vectorized, metric='euclidean')
    distance_manhattan = pairwise_distances(modified_question, qa_base_vectorized, metric='manhattan')
    cos_similarity = cosine_similarity(modified_question, qa_base_vectorized)
    
    distance_euclidean_indexes = np.argsort(distance_euclidean)
    distance_manhattan_indexes = np.argsort(distance_manhattan)
    cos_similarity_indexes = np.argsort(cos_similarity)
  
    for j in range(questions_number):
        ranked_question = qa_base["Question"][distance_euclidean_indexes[0][j]]
        if original_question == ranked_question:
            euclidean_rank.append(j+1)
            break
            
    for j in range(questions_number):
        ranked_question = qa_base["Question"][distance_manhattan_indexes[0][j]]
        if original_question == ranked_question:
            manhattan_rank.append(j+1)
            break
            
    for j in range(questions_number):
        ranked_question = qa_base["Question"][cos_similarity_indexes[0][::-1][j]]
        if original_question == ranked_question:
            cos_rank.append(j+1)
            break
            
end = time.time()
t = (end-start)/test_questions_number

print("Heavy modifications - euclidian rank:", euclidean_rank)
print("Heavy modifications - manhattan rank:", manhattan_rank)
print("Heavy modifications - cos rank:", cos_rank)

data = {'method':'loop',
         'word simplification':'stemming',
         'questions modification':'heavy',
         'metric':'euclidean',
         'average sentence rank':round(np.mean(euclidean_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'loop',
         'word simplification':'stemming',
         'questions modification':'heavy',
         'metric':'manhattan',
         'average sentence rank':round(np.mean(manhattan_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'loop',
         'word simplification':'stemming',
         'questions modification':'heavy',
         'metric':'cosine',
         'average sentence rank':round(np.mean(cos_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

Heavy modifications - euclidian rank: [955, 2688, 8250, 6475, 3, 1, 12, 12, 1, 441, 74, 372, 272, 154, 22, 8, 5, 1, 37, 2, 90]
Heavy modifications - manhattan rank: [907, 4226, 5669, 3713, 1, 1, 10, 15, 1, 326, 828, 614, 129, 82, 46, 8, 1, 1, 5, 21, 32]
Heavy modifications - cos rank: [955, 2688, 8250, 6475, 3, 1, 12, 12, 1, 441, 74, 372, 272, 154, 22, 8, 5, 1, 37, 2, 90]


/tmp/ipykernel_43013/1857327317.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/1857327317.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/1857327317.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)


## TF-IDF vectorization with lemmatization - loop method

In [12]:
qa_base_lem = cp.deepcopy(qa_base["Question"])
wnl = WordNetLemmatizer()

for i in range(questions_number):
    sentence = qa_base_lem[i]
    token_words=word_tokenize(sentence)
    lem_sentence=[]
    for word in token_words:
        lem_sentence.append(wnl.lemmatize(word))
        lem_sentence.append(" ")
        lematizated_sentence = "".join(lem_sentence)
    qa_base_lem.iloc[i] = lematizated_sentence

vectorizer = TfidfVectorizer()
qa_base_vectorized = vectorizer.fit_transform(qa_base_lem)

qlm_vectorized = vectorizer.transform(qm_base["modified question – light"])
qmm_vectorized = vectorizer.transform(qm_base["modified question – medium"])
qhm_vectorized = vectorizer.transform(qm_base["modified question – heavy"])

### Lightly modified questions with lemmatization - loop method

In [13]:
euclidean_rank = []
manhattan_rank = []
cos_rank = []

start = time.time()

for i in range(test_questions_number):
    
    original_question = qm_base["original question"][i]
    modified_question = qlm_vectorized[i]
    
    distance_euclidean = pairwise_distances(modified_question, qa_base_vectorized, metric='euclidean')
    distance_manhattan = pairwise_distances(modified_question, qa_base_vectorized, metric='manhattan')
    cos_similarity = cosine_similarity(modified_question, qa_base_vectorized)
    
    distance_euclidean_indexes = np.argsort(distance_euclidean)
    distance_manhattan_indexes = np.argsort(distance_manhattan)
    cos_similarity_indexes = np.argsort(cos_similarity)
  
    for j in range(questions_number):
        ranked_question = qa_base["Question"][distance_euclidean_indexes[0][j]]
        if original_question == ranked_question:
            euclidean_rank.append(j+1)
            break
            
    for j in range(questions_number):
        ranked_question = qa_base["Question"][distance_manhattan_indexes[0][j]]
        if original_question == ranked_question:
            manhattan_rank.append(j+1)
            break
            
    for j in range(questions_number):
        ranked_question = qa_base["Question"][cos_similarity_indexes[0][::-1][j]]
        if original_question == ranked_question:
            cos_rank.append(j+1)
            break
            
end = time.time()
t = (end-start)/test_questions_number

print("Light modifications - euclidean rank:", euclidean_rank)
print("Light modifications - manhattan rank:", manhattan_rank)
print("Light modifications - cos rank:", cos_rank)

data = {'method':'loop',
         'word simplification':'lemmatization',
         'questions modification':'light',
         'metric':'euclidean',
         'average sentence rank':round(np.mean(euclidean_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'loop',
         'word simplification':'lemmatization',
         'questions modification':'light',
         'metric':'manhattan',
         'average sentence rank':round(np.mean(manhattan_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'loop',
         'word simplification':'lemmatization',
         'questions modification':'light',
         'metric':'cosine',
         'average sentence rank':round(np.mean(cos_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

Light modifications - euclidean rank: [1, 14, 122, 703, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 5, 31, 1, 1, 1, 301]
Light modifications - manhattan rank: [1, 5, 26, 267, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 14, 1, 1, 1, 80]
Light modifications - cos rank: [1, 14, 122, 703, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 5, 31, 1, 1, 1, 301]


/tmp/ipykernel_43013/88193286.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/88193286.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/88193286.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)


### Medium modified questions with lemmatization - loop method

In [14]:
euclidean_rank = []
manhattan_rank = []
cos_rank = []

start = time.time()

for i in range(test_questions_number):

    original_question = qm_base["original question"][i]
    modified_question = qmm_vectorized[i]
    
    distance_euclidean = pairwise_distances(modified_question, qa_base_vectorized, metric='euclidean')
    distance_manhattan = pairwise_distances(modified_question, qa_base_vectorized, metric='manhattan')
    cos_similarity = cosine_similarity(modified_question, qa_base_vectorized)
    
    distance_euclidean_indexes = np.argsort(distance_euclidean)
    distance_manhattan_indexes = np.argsort(distance_manhattan)
    cos_similarity_indexes = np.argsort(cos_similarity)
  
    for j in range(questions_number):
        ranked_question = qa_base["Question"][distance_euclidean_indexes[0][j]]
        if original_question == ranked_question:
            euclidean_rank.append(j+1)
            break
            
    for j in range(questions_number):
        ranked_question = qa_base["Question"][distance_manhattan_indexes[0][j]]
        if original_question == ranked_question:
            manhattan_rank.append(j+1)
            break
            
    for j in range(questions_number):
        ranked_question = qa_base["Question"][cos_similarity_indexes[0][::-1][j]]
        if original_question == ranked_question:
            cos_rank.append(j+1)
            break
            
end = time.time()
t = (end-start)/test_questions_number

print("Medium modifications - euclidian rank:", euclidean_rank)
print("Medium modifications - manhattan rank:", manhattan_rank)
print("Medium modifications - cos rank:", cos_rank)

data = {'method':'loop',
         'word simplification':'lemmatization',
         'questions modification':'medium',
         'metric':'euclidean',
         'average sentence rank':round(np.mean(euclidean_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'loop',
         'word simplification':'lemmatization',
         'questions modification':'medium',
         'metric':'manhattan',
         'average sentence rank':round(np.mean(manhattan_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'loop',
         'word simplification':'lemmatization',
         'questions modification':'medium',
         'metric':'cosine',
         'average sentence rank':round(np.mean(cos_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

Medium modifications - euclidian rank: [8, 290, 652, 99, 11, 2, 21, 1, 3, 9, 1, 70, 1, 1, 1, 1, 5, 1, 106, 1, 161]
Medium modifications - manhattan rank: [4, 418, 248, 36, 68, 1, 15, 1, 1, 11, 1, 49, 1, 1, 1, 1, 1, 1, 114, 6, 60]
Medium modifications - cos rank: [8, 290, 652, 99, 11, 2, 21, 1, 3, 9, 1, 70, 1, 1, 1, 1, 5, 1, 106, 1, 161]


/tmp/ipykernel_43013/1945861158.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/1945861158.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/1945861158.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)


### Heavily modified questions with lemmatization - loop method

In [15]:
euclidean_rank = []
manhattan_rank = []
cos_rank = []

start = time.time()

for i in range(test_questions_number):

    original_question = qm_base["original question"][i]
    modified_question = qhm_vectorized[i]
    
    distance_euclidean = pairwise_distances(modified_question, qa_base_vectorized, metric='euclidean')
    distance_manhattan = pairwise_distances(modified_question, qa_base_vectorized, metric='manhattan')
    cos_similarity = cosine_similarity(modified_question, qa_base_vectorized)
    
    distance_euclidean_indexes = np.argsort(distance_euclidean)
    distance_manhattan_indexes = np.argsort(distance_manhattan)
    cos_similarity_indexes = np.argsort(cos_similarity)
  
    for j in range(questions_number):
        ranked_question = qa_base["Question"][distance_euclidean_indexes[0][j]]
        if original_question == ranked_question:
            euclidean_rank.append(j+1)
            break
            
    for j in range(questions_number):
        ranked_question = qa_base["Question"][distance_manhattan_indexes[0][j]]
        if original_question == ranked_question:
            manhattan_rank.append(j+1)
            break
            
    for j in range(questions_number):
        ranked_question = qa_base["Question"][cos_similarity_indexes[0][::-1][j]]
        if original_question == ranked_question:
            cos_rank.append(j+1)
            break
            
end = time.time()
t = (end-start)/test_questions_number

print("Heavy modifications - euclidian rank:", euclidean_rank)
print("Heavy modifications - manhattan rank:", manhattan_rank)
print("Heavy modifications - cos rank:", cos_rank)

data = {'method':'loop',
         'word simplification':'lemmatization',
         'questions modification':'heavy',
         'metric':'euclidean',
         'average sentence rank':round(np.mean(euclidean_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'loop',
         'word simplification':'lemmatization',
         'questions modification':'heavy',
         'metric':'manhattan',
         'average sentence rank':round(np.mean(manhattan_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'loop',
         'word simplification':'lemmatization',
         'questions modification':'heavy',
         'metric':'cosine',
         'average sentence rank':round(np.mean(cos_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

Heavy modifications - euclidian rank: [19, 2700, 1438, 1498, 7, 1, 19, 32, 2, 594, 21, 473, 1, 6, 274, 1, 5, 1, 30, 1, 28]
Heavy modifications - manhattan rank: [19, 4167, 797, 631, 5, 1, 15, 38, 1, 438, 30, 797, 1, 3, 601, 1, 1, 1, 2, 1, 13]
Heavy modifications - cos rank: [19, 2700, 1438, 1498, 7, 1, 19, 32, 2, 594, 21, 473, 1, 6, 274, 1, 5, 1, 30, 1, 28]


/tmp/ipykernel_43013/1026903693.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/1026903693.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/1026903693.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)


## TF-IDF vectorization - nearest neighbors method

In [16]:
vectorizer = TfidfVectorizer()
qa_base_vectorized = vectorizer.fit_transform(qa_base["Question"])

qlm_vectorized = vectorizer.transform(qm_base["modified question – light"])
qmm_vectorized = vectorizer.transform(qm_base["modified question – medium"])
qhm_vectorized = vectorizer.transform(qm_base["modified question – heavy"])

nbrs_euclidean = NearestNeighbors(n_neighbors=10000, metric="euclidean").fit(qa_base_vectorized)
nbrs_manhattan = NearestNeighbors(n_neighbors=10000, metric="manhattan").fit(qa_base_vectorized)
nbrs_cosine = NearestNeighbors(n_neighbors=10000, metric="cosine").fit(qa_base_vectorized)

### Lightly modified questions - nearest neighbors method

In [17]:
euclidean_rank = []
manhattan_rank = []
cos_rank = []

start = time.time()

for i in range(test_questions_number):
    
    original_question = qm_base["original question"][i]
    modified_question = qlm_vectorized[i]
    
    distances_euclidean, indices_euclidean = nbrs_euclidean.kneighbors(modified_question)
    distances_manhattan, indices_manhattan = nbrs_manhattan.kneighbors(modified_question)
    distances_cosine, indices_cosine = nbrs_cosine.kneighbors(modified_question)
    
    nbrs_list_euclidean = qa_base["Question"].iloc[indices_euclidean[0]].tolist()
    nbrs_list_manhattan = qa_base["Question"].iloc[indices_manhattan[0]].tolist()
    nbrs_list_cosine = qa_base["Question"].iloc[indices_cosine[0]].tolist()
    
    euclidean_rank.append(nbrs_list_euclidean.index(original_question)+1)
    manhattan_rank.append(nbrs_list_manhattan.index(original_question)+1)
    cos_rank.append(nbrs_list_cosine.index(original_question)+1)
    
end = time.time()
t = (end-start)/test_questions_number

print("Light modifications - euclidian rank:", euclidean_rank)
print("Light modifications - manhattan rank:", manhattan_rank)
print("Light modifications - cos rank:", cos_rank)

data = {'method':'nearest neighbors',
         'word simplification':'/',
         'questions modification':'light',
         'metric':'euclidean',
         'average sentence rank':round(np.mean(euclidean_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'nearest neighbors',
         'word simplification':'/',
         'questions modification':'light',
         'metric':'manhattan',
         'average sentence rank':round(np.mean(manhattan_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'nearest neighbors',
         'word simplification':'/',
         'questions modification':'light',
         'metric':'cosine',
         'average sentence rank':round(np.mean(cos_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

Light modifications - euclidian rank: [1, 14, 122, 703, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 5, 31, 1, 1, 1, 303]
Light modifications - manhattan rank: [1, 5, 26, 263, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 14, 1, 1, 1, 80]
Light modifications - cos rank: [1, 14, 122, 703, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 5, 31, 1, 1, 1, 303]


/tmp/ipykernel_43013/473956328.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/473956328.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/473956328.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)


### Medium modified questions - nearest neighbors method

In [18]:
euclidean_rank = []
manhattan_rank = []
cos_rank = []

start = time.time()

for i in range(test_questions_number):
    
    original_question = qm_base["original question"][i]
    modified_question = qmm_vectorized[i]
    
    distances_euclidean, indices_euclidean = nbrs_euclidean.kneighbors(modified_question)
    distances_manhattan, indices_manhattan = nbrs_manhattan.kneighbors(modified_question)
    distances_cosine, indices_cosine = nbrs_cosine.kneighbors(modified_question)
    
    nbrs_list_euclidean = qa_base["Question"].iloc[indices_euclidean[0]].tolist()
    nbrs_list_manhattan = qa_base["Question"].iloc[indices_manhattan[0]].tolist()
    nbrs_list_cosine = qa_base["Question"].iloc[indices_cosine[0]].tolist()
    
    euclidean_rank.append(nbrs_list_euclidean.index(original_question)+1)
    manhattan_rank.append(nbrs_list_manhattan.index(original_question)+1)
    cos_rank.append(nbrs_list_cosine.index(original_question)+1)
    
end = time.time()
t = (end-start)/test_questions_number

print("Medium modifications - euclidian rank:", euclidean_rank)
print("Medium modifications - manhattan rank:", manhattan_rank)
print("Medium modifications - cos rank:", cos_rank)

data = {'method':'nearest neighbors',
         'word simplification':'/',
         'questions modification':'medium',
         'metric':'euclidean',
         'average sentence rank':round(np.mean(euclidean_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'nearest neighbors',
         'word simplification':'/',
         'questions modification':'medium',
         'metric':'manhattan',
         'average sentence rank':round(np.mean(manhattan_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'nearest neighbors',
         'word simplification':'/',
         'questions modification':'medium',
         'metric':'cosine',
         'average sentence rank':round(np.mean(cos_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

Medium modifications - euclidian rank: [6, 307, 667, 101, 11, 2, 21, 1, 3, 12, 1, 69, 1, 1, 1, 1, 5, 1, 106, 1, 161]
Medium modifications - manhattan rank: [4, 423, 248, 36, 68, 1, 15, 1, 1, 11, 1, 55, 1, 1, 1, 1, 1, 1, 114, 6, 60]
Medium modifications - cos rank: [6, 307, 667, 101, 11, 2, 21, 1, 3, 12, 1, 69, 1, 1, 1, 1, 5, 1, 106, 1, 161]


/tmp/ipykernel_43013/1971219292.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/1971219292.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/1971219292.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)


### Heavily modified questions - nearest neighbors method

In [19]:
euclidean_rank = []
manhattan_rank = []
cos_rank = []

start = time.time()

for i in range(test_questions_number):
    
    original_question = qm_base["original question"][i]
    modified_question = qhm_vectorized[i]
    
    distances_euclidean, indices_euclidean = nbrs_euclidean.kneighbors(modified_question)
    distances_manhattan, indices_manhattan = nbrs_manhattan.kneighbors(modified_question)
    distances_cosine, indices_cosine = nbrs_cosine.kneighbors(modified_question)
    
    nbrs_list_euclidean = qa_base["Question"].iloc[indices_euclidean[0]].tolist()
    nbrs_list_manhattan = qa_base["Question"].iloc[indices_manhattan[0]].tolist()
    nbrs_list_cosine = qa_base["Question"].iloc[indices_cosine[0]].tolist()
    
    euclidean_rank.append(nbrs_list_euclidean.index(original_question)+1)
    manhattan_rank.append(nbrs_list_manhattan.index(original_question)+1)
    cos_rank.append(nbrs_list_cosine.index(original_question)+1)
    
end = time.time()
t = (end-start)/test_questions_number

print("Heavy modifications - euclidian rank:", euclidean_rank)
print("Heavy modifications - manhattan rank:", manhattan_rank)
print("Heavy modifications - cos rank:", cos_rank)

data = {'method':'nearest neighbors',
         'word simplification':'/',
         'questions modification':'heavy',
         'metric':'euclidean',
         'average sentence rank':round(np.mean(euclidean_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'nearest neighbors',
         'word simplification':'/',
         'questions modification':'heavy',
         'metric':'manhattan',
         'average sentence rank':round(np.mean(manhattan_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'nearest neighbors',
         'word simplification':'/',
         'questions modification':'heavy',
         'metric':'cosine',
         'average sentence rank':round(np.mean(cos_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

Heavy modifications - euclidian rank: [17, 2748, 1443, 1498, 7, 1, 19, 32, 2, 610, 21, 461, 1, 6, 299, 1, 5, 1, 30, 1, 28]
Heavy modifications - manhattan rank: [19, 4180, 801, 630, 5, 1, 15, 38, 1, 454, 33, 803, 1, 3, 628, 1, 1, 1, 2, 1, 13]
Heavy modifications - cos rank: [17, 2748, 1443, 1498, 7, 1, 19, 32, 2, 610, 21, 461, 1, 6, 299, 1, 5, 1, 30, 1, 28]


/tmp/ipykernel_43013/2098487766.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/2098487766.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/2098487766.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)


## TF-IDF vectorization with stemming - nearest neighbors method

In [20]:
qa_base_stem = cp.deepcopy(qa_base["Question"])
ps = PorterStemmer()

for i in range(questions_number):
    sentence = qa_base_stem[i]
    token_words=word_tokenize(sentence)
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(ps.stem(word))
        stem_sentence.append(" ")
        stemmed_sentence = "".join(stem_sentence)
    qa_base_stem.iloc[i] = stemmed_sentence

vectorizer = TfidfVectorizer()
qa_base_vectorized = vectorizer.fit_transform(qa_base_stem)

qlm_vectorized = vectorizer.transform(qm_base["modified question – light"])
qmm_vectorized = vectorizer.transform(qm_base["modified question – medium"])
qhm_vectorized = vectorizer.transform(qm_base["modified question – heavy"])

nbrs_euclidean = NearestNeighbors(n_neighbors=10000, metric="euclidean").fit(qa_base_vectorized)
nbrs_manhattan = NearestNeighbors(n_neighbors=10000, metric="manhattan").fit(qa_base_vectorized)
nbrs_cosine = NearestNeighbors(n_neighbors=10000, metric="cosine").fit(qa_base_vectorized)

### Lightly modified questions with stemming - nearest neighbors method

In [21]:
euclidean_rank = []
manhattan_rank = []
cos_rank = []

start = time.time()

for i in range(test_questions_number):
    
    original_question = qm_base["original question"][i]
    modified_question = qlm_vectorized[i]
    
    distances_euclidean, indices_euclidean = nbrs_euclidean.kneighbors(modified_question)
    distances_manhattan, indices_manhattan = nbrs_manhattan.kneighbors(modified_question)
    distances_cosine, indices_cosine = nbrs_cosine.kneighbors(modified_question)
    
    nbrs_list_euclidean = qa_base["Question"].iloc[indices_euclidean[0]].tolist()
    nbrs_list_manhattan = qa_base["Question"].iloc[indices_manhattan[0]].tolist()
    nbrs_list_cosine = qa_base["Question"].iloc[indices_cosine[0]].tolist()
    
    euclidean_rank.append(nbrs_list_euclidean.index(original_question)+1)
    manhattan_rank.append(nbrs_list_manhattan.index(original_question)+1)
    cos_rank.append(nbrs_list_cosine.index(original_question)+1)
    
end = time.time()
t = (end-start)/test_questions_number

print("Light modifications - euclidian rank:", euclidean_rank)
print("Light modifications - manhattan rank:", manhattan_rank)
print("Light modifications - cos rank:", cos_rank)

data = {'method':'nearest neighbors',
         'word simplification':'stemming',
         'questions modification':'light',
         'metric':'euclidean',
         'average sentence rank':round(np.mean(euclidean_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'nearest neighbors',
         'word simplification':'stemming',
         'questions modification':'light',
         'metric':'manhattan',
         'average sentence rank':round(np.mean(manhattan_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'nearest neighbors',
         'word simplification':'stemming',
         'questions modification':'light',
         'metric':'cosine',
         'average sentence rank':round(np.mean(cos_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

Light modifications - euclidian rank: [24, 42, 333, 1204, 1, 1, 1, 1, 1, 1, 2, 1, 30, 154, 1, 1, 31, 1, 88, 15, 1052]
Light modifications - manhattan rank: [11, 12, 158, 1361, 1, 1, 1, 1, 1, 1, 2, 1, 22, 82, 3, 1, 16, 1, 63, 32, 316]
Light modifications - cos rank: [24, 42, 333, 1204, 1, 1, 1, 1, 1, 1, 2, 1, 30, 154, 1, 1, 31, 1, 88, 15, 1052]


/tmp/ipykernel_43013/149841299.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/149841299.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/149841299.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)


### Medium modified questions with stemming - nearest neighbors method

In [22]:
euclidean_rank = []
manhattan_rank = []
cos_rank = []

start = time.time()

for i in range(test_questions_number):
    
    original_question = qm_base["original question"][i]
    modified_question = qmm_vectorized[i]
    
    distances_euclidean, indices_euclidean = nbrs_euclidean.kneighbors(modified_question)
    distances_manhattan, indices_manhattan = nbrs_manhattan.kneighbors(modified_question)
    distances_cosine, indices_cosine = nbrs_cosine.kneighbors(modified_question)
    
    nbrs_list_euclidean = qa_base["Question"].iloc[indices_euclidean[0]].tolist()
    nbrs_list_manhattan = qa_base["Question"].iloc[indices_manhattan[0]].tolist()
    nbrs_list_cosine = qa_base["Question"].iloc[indices_cosine[0]].tolist()
    
    euclidean_rank.append(nbrs_list_euclidean.index(original_question)+1)
    manhattan_rank.append(nbrs_list_manhattan.index(original_question)+1)
    cos_rank.append(nbrs_list_cosine.index(original_question)+1)
    
end = time.time()
t = (end-start)/test_questions_number

print("Medium modifications - euclidian rank:", euclidean_rank)
print("Medium modifications - manhattan rank:", manhattan_rank)
print("Medium modifications - cos rank:", cos_rank)

data = {'method':'nearest neighbors',
         'word simplification':'stemming',
         'questions modification':'medium',
         'metric':'euclidean',
         'average sentence rank':round(np.mean(euclidean_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'nearest neighbors',
         'word simplification':'stemming',
         'questions modification':'medium',
         'metric':'manhattan',
         'average sentence rank':round(np.mean(manhattan_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'nearest neighbors',
         'word simplification':'stemming',
         'questions modification':'medium',
         'metric':'cosine',
         'average sentence rank':round(np.mean(cos_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

Medium modifications - euclidian rank: [397, 1224, 1398, 22, 4, 1, 9, 1, 737, 12, 2, 1, 182, 288, 3, 1, 5, 1, 410, 46, 764]
Medium modifications - manhattan rank: [672, 3440, 730, 28, 4, 1, 6, 1, 216, 12, 2, 1, 101, 426, 8, 1, 1, 1, 363, 357, 267]
Medium modifications - cos rank: [397, 1224, 1398, 22, 4, 1, 9, 1, 737, 12, 2, 1, 182, 288, 3, 1, 5, 1, 410, 46, 764]


/tmp/ipykernel_43013/3195794950.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/3195794950.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/3195794950.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)


### Heavily modified questions with stemming - nearest neighbors method

In [23]:
euclidean_rank = []
manhattan_rank = []
cos_rank = []

start = time.time()

for i in range(test_questions_number):
    
    original_question = qm_base["original question"][i]
    modified_question = qhm_vectorized[i]
    
    distances_euclidean, indices_euclidean = nbrs_euclidean.kneighbors(modified_question)
    distances_manhattan, indices_manhattan = nbrs_manhattan.kneighbors(modified_question)
    distances_cosine, indices_cosine = nbrs_cosine.kneighbors(modified_question)
    
    nbrs_list_euclidean = qa_base["Question"].iloc[indices_euclidean[0]].tolist()
    nbrs_list_manhattan = qa_base["Question"].iloc[indices_manhattan[0]].tolist()
    nbrs_list_cosine = qa_base["Question"].iloc[indices_cosine[0]].tolist()
    
    euclidean_rank.append(nbrs_list_euclidean.index(original_question)+1)
    manhattan_rank.append(nbrs_list_manhattan.index(original_question)+1)
    cos_rank.append(nbrs_list_cosine.index(original_question)+1)
    
end = time.time()
t = (end-start)/test_questions_number

print("Heavy modifications - euclidian rank:", euclidean_rank)
print("Heavy modifications - manhattan rank:", manhattan_rank)
print("Heavy modifications - cos rank:", cos_rank)

data = {'method':'nearest neighbors',
         'word simplification':'stemming',
         'questions modification':'heavy',
         'metric':'euclidean',
         'average sentence rank':round(np.mean(euclidean_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'nearest neighbors',
         'word simplification':'stemming',
         'questions modification':'heavy',
         'metric':'manhattan',
         'average sentence rank':round(np.mean(manhattan_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'nearest neighbors',
         'word simplification':'stemming',
         'questions modification':'heavy',
         'metric':'cosine',
         'average sentence rank':round(np.mean(cos_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

Heavy modifications - euclidian rank: [955, 2688, 8250, 6475, 3, 1, 12, 12, 1, 441, 74, 372, 272, 154, 22, 8, 5, 1, 37, 2, 90]
Heavy modifications - manhattan rank: [907, 4226, 5669, 3713, 1, 1, 10, 15, 1, 326, 828, 614, 129, 82, 46, 8, 1, 1, 5, 21, 32]
Heavy modifications - cos rank: [955, 2688, 8250, 6475, 3, 1, 12, 12, 1, 441, 74, 372, 272, 154, 22, 8, 5, 1, 37, 2, 90]


/tmp/ipykernel_43013/586930312.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/586930312.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/586930312.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)


## TF-IDF vectorization with lemmatization - nearest neighbors method

In [24]:
qa_base_lem = cp.deepcopy(qa_base["Question"])
wnl = WordNetLemmatizer()

for i in range(questions_number):
    sentence = qa_base_lem[i]
    token_words=word_tokenize(sentence)
    lem_sentence=[]
    for word in token_words:
        lem_sentence.append(wnl.lemmatize(word))
        lem_sentence.append(" ")
        lematizated_sentence = "".join(lem_sentence)
    qa_base_lem.iloc[i] = lematizated_sentence

vectorizer = TfidfVectorizer()
qa_base_vectorized = vectorizer.fit_transform(qa_base_lem)

qlm_vectorized = vectorizer.transform(qm_base["modified question – light"])
qmm_vectorized = vectorizer.transform(qm_base["modified question – medium"])
qhm_vectorized = vectorizer.transform(qm_base["modified question – heavy"])

nbrs_euclidean = NearestNeighbors(n_neighbors=10000, metric="euclidean").fit(qa_base_vectorized)
nbrs_manhattan = NearestNeighbors(n_neighbors=10000, metric="manhattan").fit(qa_base_vectorized)
nbrs_cosine = NearestNeighbors(n_neighbors=10000, metric="cosine").fit(qa_base_vectorized)

### Lightly modified questions with lemmatization - nearest neighbors method

In [25]:
euclidean_rank = []
manhattan_rank = []
cos_rank = []

start = time.time()

for i in range(test_questions_number):
    
    original_question = qm_base["original question"][i]
    modified_question = qlm_vectorized[i]
    
    distances_euclidean, indices_euclidean = nbrs_euclidean.kneighbors(modified_question)
    distances_manhattan, indices_manhattan = nbrs_manhattan.kneighbors(modified_question)
    distances_cosine, indices_cosine = nbrs_cosine.kneighbors(modified_question)
    
    nbrs_list_euclidean = qa_base["Question"].iloc[indices_euclidean[0]].tolist()
    nbrs_list_manhattan = qa_base["Question"].iloc[indices_manhattan[0]].tolist()
    nbrs_list_cosine = qa_base["Question"].iloc[indices_cosine[0]].tolist()
    
    euclidean_rank.append(nbrs_list_euclidean.index(original_question)+1)
    manhattan_rank.append(nbrs_list_manhattan.index(original_question)+1)
    cos_rank.append(nbrs_list_cosine.index(original_question)+1)
    
end = time.time()
t = (end-start)/test_questions_number

print("Light modifications - euclidian rank:", euclidean_rank)
print("Light modifications - manhattan rank:", manhattan_rank)
print("Light modifications - cos rank:", cos_rank)

data = {'method':'nearest neighbors',
         'word simplification':'lemmatization',
         'questions modification':'light',
         'metric':'euclidean',
         'average sentence rank':round(np.mean(euclidean_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'nearest neighbors',
         'word simplification':'lemmatization',
         'questions modification':'light',
         'metric':'manhattan',
         'average sentence rank':round(np.mean(manhattan_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'nearest neighbors',
         'word simplification':'lemmatization',
         'questions modification':'light',
         'metric':'cosine',
         'average sentence rank':round(np.mean(cos_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

Light modifications - euclidian rank: [1, 14, 122, 703, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 5, 31, 1, 1, 1, 301]
Light modifications - manhattan rank: [1, 5, 26, 267, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 14, 1, 1, 1, 80]
Light modifications - cos rank: [1, 14, 122, 703, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 5, 31, 1, 1, 1, 301]


/tmp/ipykernel_43013/3598571585.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/3598571585.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/3598571585.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)


### Medium modified questions with lemmatization - nearest neighbors method

In [26]:
euclidean_rank = []
manhattan_rank = []
cos_rank = []

start = time.time()

for i in range(test_questions_number):
    
    original_question = qm_base["original question"][i]
    modified_question = qmm_vectorized[i]
    
    distances_euclidean, indices_euclidean = nbrs_euclidean.kneighbors(modified_question)
    distances_manhattan, indices_manhattan = nbrs_manhattan.kneighbors(modified_question)
    distances_cosine, indices_cosine = nbrs_cosine.kneighbors(modified_question)
    
    nbrs_list_euclidean = qa_base["Question"].iloc[indices_euclidean[0]].tolist()
    nbrs_list_manhattan = qa_base["Question"].iloc[indices_manhattan[0]].tolist()
    nbrs_list_cosine = qa_base["Question"].iloc[indices_cosine[0]].tolist()
    
    euclidean_rank.append(nbrs_list_euclidean.index(original_question)+1)
    manhattan_rank.append(nbrs_list_manhattan.index(original_question)+1)
    cos_rank.append(nbrs_list_cosine.index(original_question)+1)
    
end = time.time()
t = (end-start)/test_questions_number

print("Medium modifications - euclidian rank:", euclidean_rank)
print("Medium modifications - manhattan rank:", manhattan_rank)
print("Medium modifications - cos rank:", cos_rank)

data = {'method':'nearest neighbors',
         'word simplification':'lemmatization',
         'questions modification':'medium',
         'metric':'euclidean',
         'average sentence rank':round(np.mean(euclidean_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'nearest neighbors',
         'word simplification':'lemmatization',
         'questions modification':'medium',
         'metric':'manhattan',
         'average sentence rank':round(np.mean(manhattan_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'nearest neighbors',
         'word simplification':'lemmatization',
         'questions modification':'medium',
         'metric':'cosine',
         'average sentence rank':round(np.mean(cos_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

Medium modifications - euclidian rank: [8, 290, 652, 99, 11, 2, 21, 1, 3, 9, 1, 70, 1, 1, 1, 1, 5, 1, 106, 1, 161]
Medium modifications - manhattan rank: [4, 418, 248, 36, 68, 1, 15, 1, 1, 11, 1, 49, 1, 1, 1, 1, 1, 1, 114, 6, 60]
Medium modifications - cos rank: [8, 290, 652, 99, 11, 2, 21, 1, 3, 9, 1, 70, 1, 1, 1, 1, 5, 1, 106, 1, 161]


/tmp/ipykernel_43013/1273275245.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/1273275245.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/1273275245.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)


### Heavily modified questions with lemmatization - nearest neighbors method

In [27]:
euclidean_rank = []
manhattan_rank = []
cos_rank = []

start = time.time()

for i in range(test_questions_number):
    
    original_question = qm_base["original question"][i]
    modified_question = qhm_vectorized[i]
    
    distances_euclidean, indices_euclidean = nbrs_euclidean.kneighbors(modified_question)
    distances_manhattan, indices_manhattan = nbrs_manhattan.kneighbors(modified_question)
    distances_cosine, indices_cosine = nbrs_cosine.kneighbors(modified_question)
    
    nbrs_list_euclidean = qa_base["Question"].iloc[indices_euclidean[0]].tolist()
    nbrs_list_manhattan = qa_base["Question"].iloc[indices_manhattan[0]].tolist()
    nbrs_list_cosine = qa_base["Question"].iloc[indices_cosine[0]].tolist()
    
    euclidean_rank.append(nbrs_list_euclidean.index(original_question)+1)
    manhattan_rank.append(nbrs_list_manhattan.index(original_question)+1)
    cos_rank.append(nbrs_list_cosine.index(original_question)+1)
    
end = time.time()
t = end-start
    
end = time.time()
t = (end-start)/test_questions_number

print("Heavy modifications - euclidian rank:", euclidean_rank)
print("Heavy modifications - manhattan rank:", manhattan_rank)
print("Heavy modifications - cos rank:", cos_rank)

data = {'method':'nearest neighbors',
         'word simplification':'lemmatization',
         'questions modification':'heavy',
         'metric':'euclidean',
         'average sentence rank':round(np.mean(euclidean_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'nearest neighbors',
         'word simplification':'lemmatization',
         'questions modification':'heavy',
         'metric':'manhattan',
         'average sentence rank':round(np.mean(manhattan_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'method':'nearest neighbors',
         'word simplification':'lemmatization',
         'questions modification':'heavy',
         'metric':'cosine',
         'average sentence rank':round(np.mean(cos_rank)),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

Heavy modifications - euclidian rank: [19, 2700, 1438, 1498, 7, 1, 19, 32, 2, 594, 21, 473, 1, 6, 274, 1, 5, 1, 30, 1, 28]
Heavy modifications - manhattan rank: [19, 4167, 797, 631, 5, 1, 15, 38, 1, 438, 30, 797, 1, 3, 601, 1, 1, 1, 2, 1, 13]
Heavy modifications - cos rank: [19, 2700, 1438, 1498, 7, 1, 19, 32, 2, 594, 21, 473, 1, 6, 274, 1, 5, 1, 30, 1, 28]


/tmp/ipykernel_43013/3858583543.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/3858583543.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_43013/3858583543.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)


## View of results

In [28]:
results.to_csv("tf-idf - results.csv",index=False)

results.head(60)

,method,word simplification,questions modification,metric,average sentence rank,average execution time [s]
0,loop,/,light,euclidean,57,0.0293
1,loop,/,light,manhattan,19,0.0293
2,loop,/,light,cosine,57,0.0293
3,loop,/,medium,euclidean,70,0.0286
4,loop,/,medium,manhattan,50,0.0286
5,loop,/,medium,cosine,70,0.0286
6,loop,/,heavy,euclidean,344,0.0513
7,loop,/,heavy,manhattan,363,0.0513
8,loop,/,heavy,cosine,344,0.0513
9,loop,stemming,light,euclidean,142,0.0334
